In [1]:
from typing import Tuple, List, Dict

import pandas as pd
import folium

pd.set_option('display.precision', 2)


df_sites = pd.DataFrame(
    [['hotel',              48.8527, 2.3542],
     ['Sacre Coeur',        48.8867, 2.3431],
     ['Louvre',             48.8607, 2.3376],
     ['Montmartre',         48.8872, 2.3388],
     ['Port de Suffren',    48.8577, 2.2902],
     ['Arc de Triomphe',    48.8739, 2.2950],
     ['Av. Champs Élysées', 48.8710, 2.3036],
     ['Notre Dame',         48.8531, 2.3498],
     ['Tour Eiffel',        48.8585, 2.2945]],
    columns=pd.Index(['site', 'latitude', 'longitude'], name='paris')
)

df_sites

paris,site,latitude,longitude
0,hotel,48.85,2.35
1,Sacre Coeur,48.89,2.34
2,Louvre,48.86,2.34
3,Montmartre,48.89,2.34
4,Port de Suffren,48.86,2.29
5,Arc de Triomphe,48.87,2.29
6,Av. Champs Élysées,48.87,2.30
7,Notre Dame,48.85,2.35
8,Tour Eiffel,48.86,2.29


In [2]:
avg_location = df_sites[['latitude', 'longitude']].mean()
map_paris = folium.Map(location=avg_location, zoom_start=13)

In [3]:
for site in df_sites.itertuples():
    marker = folium.Marker(location=(site.latitude, site.longitude),
                           tooltip=site.site)
    marker.add_to(map_paris)

map_paris

In [4]:
df_route = df_sites.copy()
df_route.index.name = 'visit_order'

df_route

paris,site,latitude,longitude
visit_order,,,
0,hotel,48.85,2.35
1,Sacre Coeur,48.89,2.34
2,Louvre,48.86,2.34
3,Montmartre,48.89,2.34
4,Port de Suffren,48.86,2.29
5,Arc de Triomphe,48.87,2.29
6,Av. Champs Élysées,48.87,2.30
7,Notre Dame,48.85,2.35
8,Tour Eiffel,48.86,2.29


In [5]:
df_route_segments = df_route.join(
    df_route.shift(-1),  # map each stop to its next stop
    rsuffix='_next'
).dropna()  # last stop has no "next one", so drop it

df_route_segments

paris,site,latitude,longitude,site_next,latitude_next,longitude_next
visit_order,,,,,,
0,hotel,48.85,2.35,Sacre Coeur,48.89,2.34
1,Sacre Coeur,48.89,2.34,Louvre,48.86,2.34
2,Louvre,48.86,2.34,Montmartre,48.89,2.34
3,Montmartre,48.89,2.34,Port de Suffren,48.86,2.29
4,Port de Suffren,48.86,2.29,Arc de Triomphe,48.87,2.29
5,Arc de Triomphe,48.87,2.29,Av. Champs Élysées,48.87,2.30
6,Av. Champs Élysées,48.87,2.30,Notre Dame,48.85,2.35
7,Notre Dame,48.85,2.35,Tour Eiffel,48.86,2.29


In [6]:
map_paris = folium.Map(location=avg_location, zoom_start=13)

for stop in df_route_segments.itertuples():
    # marker for current stop
    marker = folium.Marker(location=(stop.latitude, stop.longitude),
                           tooltip=stop.site)
    # line for the route segment connecting current to next stop
    line = folium.PolyLine(
        locations=[(stop.latitude, stop.longitude), 
                   (stop.latitude_next, stop.longitude_next)],
        tooltip=f"{stop.site} to {stop.site_next}",
    )
    # add elements to the map
    marker.add_to(map_paris)
    line.add_to(map_paris)

# maker for last stop wasn't added in for loop, so adding it now 
folium.Marker(location=(stop.latitude_next, stop.longitude_next),
              tooltip=stop.site_next).add_to(map_paris);

map_paris

In [7]:
map_paris = folium.Map(location=avg_location, zoom_start=13)

for stop in df_route_segments.itertuples():
    initial_stop = stop.Index == 0
    # icon for current stop
    icon = folium.Icon(icon='home' if initial_stop else 'info-sign', 
                       color='cadetblue' if initial_stop else 'red')
    # marker for current stop
    marker = folium.Marker(location=(stop.latitude, stop.longitude),
                           icon=icon, tooltip=stop.site)
    # line for the route segment connecting current to next stop
    line = folium.PolyLine(
        locations=[(stop.latitude, stop.longitude), 
                   (stop.latitude_next, stop.longitude_next)],
        tooltip=f"{stop.site} to {stop.site_next}",
    )
    # add elements to the map
    marker.add_to(map_paris)
    line.add_to(map_paris)

# When for loop ends, the stop variable has the second-to-last 
# stop in the route, so the marker for the last stop is missing 
# We add it now using the "next" columns of the last row
folium.Marker(
    location=(stop.latitude_next, stop.longitude_next),
    tooltip=stop.site_next, 
    icon = folium.Icon(icon='info-sign', color='red')
).add_to(map_paris);

map_paris

In [8]:
from geopy.distance import geodesic

_Location = Tuple[float, float]


def ellipsoidal_distance(point1: _Location, point2: _Location) -> float:
    """Calculate ellipsoidal distance (in meters) between point1 and 
    point2 where each point is represented as a tuple (lat, lon)"""
    return geodesic(point1, point2).meters

In [9]:
df_route_segments['distance_seg'] = df_route_segments.apply(
    lambda stop: ellipsoidal_distance(
        (stop.latitude, stop.longitude), 
        (stop.latitude_next, stop.longitude_next)), 
    axis=1
)

df_route_segments

paris,site,latitude,longitude,site_next,latitude_next,longitude_next,distance_seg
visit_order,,,,,,,
0,hotel,48.85,2.35,Sacre Coeur,48.89,2.34,3867.74
1,Sacre Coeur,48.89,2.34,Louvre,48.86,2.34,2919.40
2,Louvre,48.86,2.34,Montmartre,48.89,2.34,2948.31
3,Montmartre,48.89,2.34,Port de Suffren,48.86,2.29,4844.92
4,Port de Suffren,48.86,2.29,Arc de Triomphe,48.87,2.29,1835.65
5,Arc de Triomphe,48.87,2.29,Av. Champs Élysées,48.87,2.30,708.53
6,Av. Champs Élysées,48.87,2.30,Notre Dame,48.85,2.35,3931.12
7,Notre Dame,48.85,2.35,Tour Eiffel,48.86,2.29,4102.26


In [10]:
map_paris = folium.Map(location=avg_location, zoom_start=13)

for stop in df_route_segments.itertuples():
    initial_stop = stop.Index == 0
    # marker for current stop
    icon = folium.Icon(icon='home' if initial_stop else 'info-sign', 
                       color='cadetblue' if initial_stop else 'red')
    marker = folium.Marker(
        location=(stop.latitude, stop.longitude),
        icon=icon, 
        # display the name and stop number at each site's marker
        tooltip=f"<b>Name</b>: {stop.site} <br>" \
              + f"<b>Stop number</b>: {stop.Index} <br>"
    )
    # line for the route segment connecting current to next stop
    line = folium.PolyLine(
        locations=[(stop.latitude, stop.longitude), 
                   (stop.latitude_next, stop.longitude_next)],
        # display the start, end, and distance of each segment
        tooltip=f"<b>From</b>: {stop.site} <br>" \
              + f"<b>To</b>: {stop.site_next} <br>" \
              + f"<b>Distance</b>: {stop.distance_seg:.0f} m",
    )
    # add elements to the map
    marker.add_to(map_paris)
    line.add_to(map_paris)

# add route's last marker, as it wasn't included in for loop
folium.Marker(
    location=(stop.latitude_next, stop.longitude_next),
    tooltip=f"<b>Name</b>: {stop.site_next} <br>" \
          + f"<b>Stop number</b>: {stop.Index + 1} <br>", 
    icon = folium.Icon(icon='info-sign', color='red')
).add_to(map_paris);

map_paris

In [11]:
def _make_route_segments_df(df_route: pd.DataFrame) -> pd.DataFrame:
    """Given a dataframe whose rows are ordered stops in a route, 
    and where the index has integers representing the visit order of those
    stops, return a dataframe having new columns with the information of 
    each stop's next site"""
    df_route_segments = df_route.join(
        df_route.shift(-1),  # map each stop to its next
        rsuffix='_next').dropna()

    df_route_segments['distance_seg'] = df_route_segments.apply(
        lambda stop: ellipsoidal_distance(
            (stop.latitude, stop.longitude), 
            (stop.latitude_next, stop.longitude_next)
        ), axis=1
    )
    return df_route_segments


def plot_route_on_map(df_route: pd.DataFrame) -> folium.Map:
    """Takes a dataframe of a route and displays it on a map, adding 
    a marker for each stop and a line for each pair of consecutive 
    stops"""
    df_route_segments = _make_route_segments_df(df_route)
    
    # create empty map
    avg_location = df_route[['latitude', 'longitude']].mean()
    map_route = folium.Map(location=avg_location, zoom_start=13)

    for stop in df_route_segments.itertuples():
        initial_stop = stop.Index == 0
        # marker for current stop
        icon = folium.Icon(icon='home' if initial_stop else 'info-sign', 
                           color='cadetblue' if initial_stop else 'red')
        marker = folium.Marker(
            location=(stop.latitude, stop.longitude),
            icon=icon, 
            tooltip=f"<b>Name</b>: {stop.site} <br>" \
                  + f"<b>Stop number</b>: {stop.Index} <br>"
        )
        # line for the route segment connecting current to next stop
        line = folium.PolyLine(
            locations=[(stop.latitude, stop.longitude), 
                       (stop.latitude_next, stop.longitude_next)],
            # add to each line its start, end, and distance
            tooltip=f"<b>From</b>: {stop.site} <br>" \
                  + f"<b>To</b>: {stop.site_next} <br>" \
                  + f"<b>Distance</b>: {stop.distance_seg:.0f} m",
        )
        # add elements to the map
        marker.add_to(map_route)
        line.add_to(map_route)
    
    # When for loop ends, the stop variable has the second-to-last stop in 
    # the route, so the marker for the last stop is missing, and we add it 
    # now using the "next" columns of the last row
    folium.Marker(
        location=(stop.latitude_next, stop.longitude_next),
        tooltip=f"<b>Name</b>: {stop.site_next} <br>" \
              + f"<b>Stop number</b>: {stop.Index + 1} <br>", 
        icon = folium.Icon(icon='info-sign', color='red')
    ).add_to(map_route)
    
    return map_route

In [12]:
df_route_closed = pd.concat(
   [df_route, df_route.head(1)], ignore_index=True
)
df_route_closed.index.name = df_route.index.name

df_route_closed

paris,site,latitude,longitude
visit_order,,,
0,hotel,48.85,2.35
1,Sacre Coeur,48.89,2.34
2,Louvre,48.86,2.34
3,Montmartre,48.89,2.34
4,Port de Suffren,48.86,2.29
5,Arc de Triomphe,48.87,2.29
6,Av. Champs Élysées,48.87,2.30
7,Notre Dame,48.85,2.35
8,Tour Eiffel,48.86,2.29


In [13]:
plot_route_on_map(df_route_closed)